# ShOESS experiment sensitivity

This widget draws the sensitivity of the ShOESS (Short baseline Oscillations at the ESS) experiment, assuming a cylinder-shaped detector filled with LAB doped with Gd at 0.1%.

In [51]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import csv
import array

from math import sqrt

from scipy import interpolate
from scipy.constants import Avogadro
from scipy.stats import poisson, norm, rv_continuous

from IPython.display import display_markdown

from particle import literals as lp

plt.rcParams.update({
    "font.family": "serif",
    # "text.usetex": True
})

In [3]:
miniboone = np.loadtxt("data/numu_disapp.csv", delimiter=",")
microboone = np.loadtxt("data/microboone.csv", delimiter=",")
gallex = np.loadtxt("data/gallex.csv", delimiter=",")
gallex2 = np.loadtxt("data/gallex2.csv", delimiter=",")
reno = np.loadtxt("data/nue_disapp.csv", delimiter=",")
lsnd1 = np.loadtxt("data/lsnd1.csv", delimiter=",")
lsnd2 = np.loadtxt("data/lsnd2.csv", delimiter=",")
lsnd3 = np.loadtxt("data/lsnd3.csv", delimiter=",")

e_numu = (lp.pi_plus.mass**2-lp.mu_plus.mass**2)/(2*lp.pi_plus.mass)

In [4]:
class antinumu_dar(rv_continuous):
    """$\bar{\nu}_{\mu}$ spectrum from pion decay at rest
    https://arxiv.org/abs/1911.00762
    """
    def _pdf(self, e_nu):
        pdf = 64./lp.mu_plus.mass * ((e_nu/lp.mu_plus.mass)**2 * (3./4 - e_nu/lp.mu_plus.mass))
        if isinstance(pdf, np.ndarray):
            pdf[e_nu>lp.mu_plus.mass/2] = 0
        else:
            if e_nu>lp.mu_plus.mass/2:
                return 0
        return pdf
    
class nue_dar(rv_continuous):
    """$\nu_e$ spectrum from pion decay at rest
    https://arxiv.org/abs/1911.00762
    """
    def _pdf(self, e_nu):
        pdf = 192./lp.mu_plus.mass * ((e_nu/lp.mu_plus.mass)**2 * (1./2 - e_nu/lp.mu_plus.mass))
        
        if isinstance(pdf, np.ndarray):
            pdf[e_nu>lp.mu_plus.mass/2] = 0
        else:
            if e_nu>lp.mu_plus.mass/2:
                return 0
        return pdf    

In [5]:
def appearance_prob(sin2theta2, deltam2, e, l):
    return sin2theta2*(np.sin(1.27*deltam2*l/e)**2)

def disappearance_prob(sin2theta2, deltam2, e, l):
    return 1 - sin2theta2*np.sin(1.27*deltam2*l/e)**2

In [30]:
style = {"description_width": "initial"}
layout = widgets.Layout(width="500px")
detector_efficiency = widgets.FloatSlider(
    description="Detector efficiency",
    style=style,
    layout=layout,
    min=0,
    max=1,
    value=0.5,
    continuous_update=False
)

beam_on_efficiency = widgets.FloatSlider(
    description="Beam-on efficiency",
    style=style,
    layout=layout,
    min=0,
    max=1,
    value=0.5,
    continuous_update=False
)

years = widgets.IntSlider(
    description="Years",
    min=1,
    value=1,
    max=6,
    layout=layout,
    style=style,
    continuous_update=False
)

ess_flux = widgets.FloatSlider(
    description="ESS yearly flux",
    style=style,
    layout=layout,
    min=1e21,
    max=1e23,
    value=8.5e22,
    readout_format='.2e',
    continuous_update=False
)

distance = widgets.IntSlider(
    description="Distance from target [m]",
    style=style,
    layout=layout,
    min=10,
    max=80,
    value=25,
    continuous_update=False
)

length = widgets.FloatSlider(
    description="Detector length [m]",
    style=style,
    layout=layout,
    min=1,
    value=4,
    max=10,
    continuous_update=False
)


radius = widgets.FloatSlider(
    description="Detector radius [m]",
    style=style,
    layout=layout,
    min=0.5,
    value=2,
    max=5,
    continuous_update=False
)

LAB = 2
MINERAL_OIL = 1

material = widgets.Dropdown(
    options=[('Mineral oil', MINERAL_OIL), ('Liquid scintillator', LAB)],
    value=LAB,
    style=style,
    description='Material:',
)

In [31]:
# from ROOT import TFeldmanCousins

In [32]:
class ConfLimitsCalculator(object):
    """dosctring for class ConfLimitsCalculator"""

    def __init__(self, CL):
        self.CL = CL

    def UpperLimit(self, obs, bkg):
        pass

    def LowerLimit(self, n, b):
        pass

    def AverageUpperLimit(self):
        pass        


class FeldmanCousins(ConfLimitsCalculator):
    """docstring for class FeldmanCousins"""

    def __init__(self, CL=90):
        super(FeldmanCousins, self).__init__(CL)
        # self.FC = TFeldmanCousins(self.CL/100.) # Requires CL as a fraction
        # self.FC.SetMuMax(500.)

    def UpperLimit(self, obs, bkg):
        """
        obs: observed number of events
        bkg: mean number of background events
        """
        return self.FC.CalculateUpperLimit(obs,bkg)

    def LowerLimit(self, obs, bkg):
        """
        obs: observed number of events
        bkg: mean number of background events
        """
        return self.FC.CalculateLowerLimit(obs,bkg)

    def AverageUpperLimit(self, bkg):
        """
        For a number of events b, compute the average upper limit. That is:
        UL = Sum Po(n;b) * Upper (n,b)
        """
        ### The Poisson distribution, Po(n;b), is defined only for b>0.
        ### Therefore, this method returns 0 if bkg is negative, and uses
        ### a number close to 0 for the computation if bkg=0.
        if bkg<0.:
            return 0.
        elif bkg==0.:
            bkg=1.E-5

        ### We'll compute the sum in the range [-5sigma, +5sigma] around
        ### the mean, where sigma is the standard deviation of the Poisson
        ### distribution.
        sigma = np.sqrt(bkg)
        nmin = max(0,  int(bkg-5.*sigma))   # Use 0 if nmin<0
        nmax = max(20, int(bkg+5.*sigma)+1) # Use at least 20 for low means
        #print "nmin=%f, nmax=%f" % (nmin,nmax)

        po = poisson(bkg)
        UL = 0.

        for i in range(nmin, nmax):
            pmf = po.pmf(i)
            ul = self.FC.CalculateUpperLimit(i, bkg)
            #print "i=%i, Po(i)=%f, U(i,b)=%f" % (i, pmf, ul)
            UL += po.pmf(i) * self.FC.CalculateUpperLimit(i,bkg)

        return UL


class FCMemoizer(FeldmanCousins):
    """docstring for FCMemoizer"""

    def __init__(self, CL=90):
        super(FCMemoizer, self).__init__(CL)
        self.AULs = 0

    def ComputeTableAverageUpperLimits(self, bmin, bmax, step, filename):
        """Compute a lookup table of average upper limits."""

        writer = csv.writer(open(filename, 'w'))

        brange = np.arange(bmin, bmax, step)

        for b in tqdm(brange):
            UL = super(FCMemoizer, self).AverageUpperLimit(b)
            writer.writerow([b,UL])

    def AverageUpperLimit(self, bkg):
        """Use tabulated data (or for large values of bkg, a mathematical 
        function extracted from a fit to the data) to speed up the computation
        of the Feldman-Cousins average upper limit for a given background
        prediction."""

        if bkg < 0:
            return 0.
        elif bkg < 100.:
            return self.AULs(bkg)
        else:
            return self.FitFunction(bkg)

    def ReadTableAverageUpperLimits(self, filename=''):

        if filename == '':
            filename = DATA_PATH + 'FC' + str(self.CL) + '.dat'

        try:
            reader = csv.reader(open(filename, 'r'))
            xs = array.array('f')
            ys = array.array('f')
            for row in reader:
                xs.append(float(row[0]))
                ys.append(float(row[1]))
            self.AULs = interpolate.interp1d(xs, ys)
        except IOError:
            raise IOError("ERROR: Memoizer file not found!\n")
            # raise
        
    def FitFunction(self, x):
        """Returns a value for the Feldman-Cousins average upper limit
        using a mathematical function extracted from a fit to the data."""
        if self.CL==90:
            return 1.225 + 1.7312 * np.sqrt(x)
        elif self.CL==norm.cdf(5)*100:
            return 10.7429333 + 5.01733764 * np.sqrt(x)
        elif self.CL==norm.cdf(3)*100:
            return 4.2444014 + 3.20614678 * np.sqrt(x)
        else:
            raise NotImplementedError(f"Fit not implement for CL {self.CL}")

In [46]:
ESS_FLUX_YEAR = 8.5e22
ESS_NUE_CONTAMINATION = 1/487.6
deltam2s = np.logspace(-2,2,100)
sin2theta2s = np.logspace(-4,0,100)
es = np.linspace(0.5, lp.mu_minus.mass/2, int(np.ceil(lp.mu_minus.mass/2*2)))

CH2_MASS = 2.32e-23 #g
CH2_DENSITY = 0.86 #g/cm3

LAB_DENSITY = 0.853 #g/cm3
LAB_MASS = 243/Avogadro #g
LAB_C_ATOMS = 16
LAB_H_ATOMS = 26


def draw_poisson_contour(x, y, ax, sig, bkg, levels, colors):
    h = ax.contour(x,
                   y, 
                   poisson.sf(k=sig+bkg, mu=bkg), 
                   levels,
                   colors=colors)
    
    return h

def draw_fc_contour(x, y, ax, sig, bkg, levels, colors):
    for color, level in zip(colors, levels):
        fcm = FCMemoizer((1-level)*100)
        ul = fcm.FitFunction(bkg)
        h = ax.contour(x,
                       y, 
                       sig-ul,
                       0,
                       colors=color)
    
    return h, sig-ul


def gui(ess_flux, detector_efficiency, years, distance, length, radius):
    flux_norm = detector_efficiency
    flux_norm /= 4 * np.pi * (distance*100)**2
    flux_norm *= years * ess_flux

    volume = np.pi*radius*radius*length*1e6 #cm3
    mass = volume*LAB_DENSITY
    gd_atoms = mass * 0.1/100 / 2.61e-22 
    # neutron_rate = 2e-3 # neutrons * s / cm^2
    # print("beam neutron interactions per spill", gd_atoms * 48800e-28 * 2e-3 * 2.8e-3 )

    n_c_atoms = LAB_DENSITY * volume / LAB_MASS * LAB_C_ATOMS
    n_h_atoms = LAB_DENSITY * volume / LAB_MASS * LAB_H_ATOMS
    
    display_markdown(f"""ESS total neutrino flux: {ess_flux * years:.2e} / flavor \n
Scintillator mass: {volume * LAB_DENSITY / 1000:.2e} kg\n
Number of hydrogen atoms: {n_h_atoms:.2e}\n
Number of carbon atoms: {n_c_atoms:.2e}""", raw=True)

    ls = np.linspace(distance-length/2,distance+length/2,5)
    s_antinue, d_antinue, e_antinue, l_antinue = np.meshgrid(sin2theta2s, deltam2s, es[es>20], ls, indexing='ij')
    s, d, e, l = np.meshgrid(sin2theta2s, deltam2s, es, ls, indexing='ij')

    app_antinue_probs = np.mean(np.average(appearance_prob(s_antinue, d_antinue, e_antinue, l_antinue), 
                                           axis=2, weights=antinumu_dar().pdf(es[es>20])), 
                                axis=2)
    app_nue_probs = np.mean(appearance_prob(s, d, e_numu, l), axis=(2,3))
    disapp_numu_probs = np.mean(disappearance_prob(s, d, e_numu, l), axis=(2,3))
    disapp_nue_probs = np.mean(np.average(disappearance_prob(s, d, e, l), axis=2, weights=nue_dar().pdf(es)), axis=2)

    fig, ax = plt.subplots(1,2, layout='constrained', figsize=(10,5))
    for i in range(2):
        ax[i].set_xscale("log")
        ax[i].set_yscale("log")
        ax[i].grid()
        ax[i].set_ylabel(r"$\Delta m^2_{41}$ [eV$^2$]", fontsize='xx-large')
        ax[i].tick_params(axis='both', which='major', labelsize='x-large')

    # https://arxiv.org/pdf/hep-ex/0203021.pdf
    ibd_p_xsec_bkg = 72.0e-42
    ibd_c_xsec_bkg = 7.4e-42
    tot_bkg_events = ibd_p_xsec_bkg * n_h_atoms + ibd_c_xsec_bkg * n_c_atoms

    ibd_p_xsec_sig = 93.5e-42
    ibd_c_xsec_sig = 8.5e-42
    tot_sig_events = ibd_p_xsec_sig * n_h_atoms + ibd_c_xsec_sig * n_c_atoms
    sig_antinue_app = tot_sig_events*flux_norm*app_antinue_probs 
    cosmic_bkg = 82 * np.pi * radius * radius/(np.pi * 2 * 2)
    bkg_antinue_app = tot_bkg_events*flux_norm*ESS_NUE_CONTAMINATION+82*years+77*years
    miniboone_best_point = (0.807, 0.043)
    mb = ax[0].scatter(*miniboone_best_point, zorder=999, c='tab:blue', label="MiniBooNE best fit")
    sig_events_miniboone = tot_sig_events*flux_norm*np.mean(appearance_prob(0.807, 0.043, e_numu, l))
    display_markdown(r"### $\bar{\nu}_e$ appearance", raw=True)
    display_markdown(r"Signal events at $(\sin 2\theta_{\mu e}, \Delta m^2)=(%g, %g~\mathrm{eV}^2)$: %.2e" % (*miniboone_best_point, sig_events_miniboone),  raw=True)
    display_markdown("Background events: %.2e" % bkg_antinue_app,  raw=True)
    s_2d = np.mean(s,axis=(2,3))
    d_2d = np.mean(d,axis=(2,3))

    levels = [1-norm.cdf(5), 1-norm.cdf(3), 0.10]
    colors = ['gold', 'lightcoral', 'mediumseagreen']
    
    draw_fc_contour(s_2d, d_2d, ax[0], sig_antinue_app, bkg_antinue_app, levels, colors)
    # draw_poisson_contour(s_2d, d_2d, ax[0], sig_antinue_app, bkg_antinue_app, levels, colors)

    l1, = ax[0].plot(np.nan, np.nan, color='gold', label=r'ShOESS $5\sigma$ CL')
    l2, = ax[0].plot(np.nan, np.nan, color='lightcoral', label=r'ShOESS $3\sigma$ CL')
    l3, = ax[0].plot(np.nan, np.nan, color='mediumseagreen', label=r'ShOESS 90\% CL')

    ax[0].set_xlabel(r"$\sin^22 \theta_{\mu e}$", fontsize='xx-large')
    ax[0].set_ylim(1e-2, 1e2)
    ax[0].set_title(r"$\bar{\nu}_{\mu}\rightarrow\bar{\nu}_e$ (%i year%s)" % (years, 's' if years > 1 else ''))

    m, = ax[0].plot(microboone[:,0][np.argsort(microboone[:,1])], microboone[:,1][np.argsort(microboone[:,1])], ls='--')
    l4, = ax[0].fill(lsnd1[:,0], lsnd1[:,1], c='lightblue')
    l5, = ax[0].fill(lsnd2[:,0], lsnd2[:,1], c='lightblue')
    l6, = ax[0].fill(lsnd3[:,0], lsnd3[:,1], c='lightblue')

    ax[0].legend([l1, l2, l3, m, mb, l4, l5, l6], 
              [r'ShOESS 5$\sigma$ CL', 'ShOESS 3$\sigma$ CL', 'ShOESS 90\% CL',
               r"MicroBooNE 95\% CL ($\nu_e$ app.)", "MiniBooNE best fit", "LSND 99\% CL (allowed)"],
             loc='upper right')

#     nc_xsec = 3.2e-42 # Phys. Lett. B423 (1998) 15
#     tot_bkg_events = nc_xsec * n_c_atoms
#     bkg_dis_numu = tot_bkg_events*flux_norm*disapp_numu_probs
#     sig_dis_numu = tot_bkg_events*flux_norm*(1-disapp_numu_probs)    
#     fig, ax = plt.subplots(1,1, layout='constrained', figsize=(5,5))

#     draw_fc_contour(s_2d, d_2d, ax, sig_dis_numu, bkg_dis_numu, levels, colors)

#     ax.set_xlabel(r"$\sin^22 \theta_{\mu \mu}$")
#     ax.set_xlabel(r"$\sin^22 \theta_{\mu \mu}$")
#     ax.set_ylim(1e-2, 50)
#     ax.set_xlim(1e-2, 1)
#     ax.set_xscale("log")
#     ax.set_yscale("log")
#     ax.grid()

#     ax.set_title(r"$\nu_{\mu}\rightarrow \nu_s$ (%i year%s)" % (years, 's' if years > 1 else ''))

#     ax.plot(miniboone[:,0][np.argsort(miniboone[:,1])], miniboone[:,1][np.argsort(miniboone[:,1])], ls='--', label='MiniBooNE+SciBooNE 90% CL')
#     ax.legend()
            
    cc_xsec = 8.9e-42 # https://arxiv.org/pdf/hep-ex/0105068.pdf
    tot_bkg_events = cc_xsec * n_c_atoms
    bkg_dis_nue = tot_bkg_events*flux_norm*disapp_nue_probs*0.3+cosmic_bkg*years*2+25*years
    sig_dis_nue = tot_bkg_events*flux_norm*(1-disapp_nue_probs)*0.3
    gallex_best_point = (0.38, 7.3)
    sig_best_point = (1-np.mean(np.average(disappearance_prob(*gallex_best_point, e, l), axis=2, weights=nue_dar().pdf(es))))*tot_bkg_events*flux_norm*0.30
    bkg_best_point = np.mean(np.average(disappearance_prob(*gallex_best_point, e, l), axis=2, weights=nue_dar().pdf(es)))*tot_bkg_events*flux_norm*0.30+cosmic_bkg*years*2+25*years
    display_markdown(r"### $\nu_e$ disappearance", raw=True)
    display_markdown(r"Signal events at $(\sin 2\theta_{ee}, \Delta m^2)=(%g, %g~\mathrm{eV}^2)$: %.2e" % (*gallex_best_point, sig_best_point), raw=True)
    display_markdown("Background events: %.2e" % bkg_best_point, raw=True)

    best = ax[1].scatter(0.38, 7.3, zorder=999, c='tab:blue', label="GALLEX+SAGE+BEST best fit")

    draw_fc_contour(s_2d, d_2d, ax[1], sig_dis_nue, bkg_dis_nue, levels, colors)

    ax[1].set_xlabel(r"$\sin^22 \theta_{ee}$", fontsize='xx-large')
    ax[1].set_ylim(1e-1, 40)
    ax[1].set_xlim(2e-3, 1)

    ax[1].set_title(r"$\nu_e\rightarrow \nu_s$ (%i year%s)" % (years, 's' if years > 1 else ''))

    g, = ax[1].fill(gallex[:,0], gallex[:,1], label='GALLEX+SAGE+BEST\n95\% CL (allowed)', c='lightblue')
    r, = ax[1].plot(reno[:,0][np.argsort(reno[:,1])], reno[:,1][np.argsort(reno[:,1])], ls='--', label='RENO+NEOS 95\% CL')
    ax[1].fill(gallex2[:,0], gallex2[:,1], c='lightblue')
    ax[1].legend(
        [l1, l2, l3, g, best, r], 
        [r'ShOESS 5$\sigma$ CL', 'ShOESS 3$\sigma$ CL', 'ShOESS 90\% CL',
         'GALLEX+SAGE+BEST\n95\% CL (allowed)', "GALLEX+SAGE+BEST\nbest fit", 'RENO+NEOS 95\% CL'],
        loc='upper left')

    fig.savefig("disapp.pdf", transparent=True)


In [49]:
# gui(ESS_FLUX_YEAR*2, 0.5, 1, 25, 4, 2)

In [50]:
out = widgets.interactive_output(
    gui,
    {
        "ess_flux": ess_flux,
        "detector_efficiency": detector_efficiency,
        "years": years,
        "distance": distance,
        "length": length,
        "radius": radius
    },
)
ui = widgets.VBox(
    [
        ess_flux,
        detector_efficiency,
        years,
        distance,
        length,
        radius
    ]
)

display(ui, out)


Output()

<!-- ## $\bar{\nu}_{\mu}\rightarrow\bar{\nu}_{e}$ appearance 
This happens through inverse beta decay on C and free protons

\begin{align}
    \bar{\nu}_{e} + ^{12}C &\rightarrow e^+ + ^{11}B + n\\
    \bar{\nu}_{e} + p &\rightarrow e^+ + n
\end{align}

followed by neutron capture and the emission of a 2.2 MeV $\gamma$.


## $\nu_{\mu}$ disappearance
Here we look for a deficit of muon neutrinos through the neutral current channel

\begin{equation}
    \nu_{\mu} + ^{12}C \rightarrow \nu_{\mu} + ^{12}C^*
\end{equation}

and the subsequent emission of 15.11 MeV $\gamma$.

## $\nu_e$ disappearance
Here we look for a deficit of electron neutrinos through the charged current channel

\begin{equation}
    \nu_e + ^{12}C \rightarrow e^- + ^{12}N_{gs}
\end{equation}

followed by the $\beta^+$ decay of $^{12}N_{gs}$ into $^{12}C$ with the emission of one positron. -->